# Visualizing Data
They say that a picture is worth a thousand words and often looking at data in a chart reveals a whole lot more than by merely looking at numbers in a table.  In the previous examples you have already seen how easy it is to draw charts in F#.  We did that with the [XPlot](https://fslab.org/XPlot/) library (which is built into Jupyter Notebook).  XPlot is a F# friendly wrapper on top of [Google Plots](https://fslab.org/XPlot/google-charts.html) and [Plotly](https://fslab.org/XPlot/plotly.html).  

In general you create a list of tuples as input to the charts.  As you can read in the documentation for each chart, what goes into those tuples varies between the chart types.  For example, this example for the Bar chart shows how you create a list of tuples for each of the to data series and then pass them on as a list of lists of tuples to the chart.

In [1]:
let layout = Layout(title = "Basic Bar Chart")

// List of tuples
let data1 = ["giraffes", 20; "orangutans", 14; "monkeys", 23]

// Another list of tuples
let data2 = ["giraffes", 15; "orangutans", 16; "monkeys", 21]

// A list of lists
[data1; data2]
|> Chart.Bar
|> Chart.WithLayout layout
|> Chart.WithHeight 500
|> Chart.WithWidth 700

You can also use a `Plot` to contain other chart types. 

In [2]:
let groupedTrace1 =
    Bar(
        x = ["giraffes"; "orangutans"; "monkeys"],
        y = [20; 14; 23],
        name= "SF Zoo"            
    )

let groupedTrace2 =
    Bar(
        x = ["giraffes"; "orangutans"; "monkeys"],
        y = [12; 18; 29],
        name = "LA Zoo"
    )

let groupedLayout = Layout(barmode = "group")

[groupedTrace1; groupedTrace2]
|> Chart.Plot
|> Chart.WithLayout groupedLayout
|> Chart.WithHeight 500
|> Chart.WithWidth 700


I recommend reading the documentation for more examples on how to use XPlot.

Lets see if we can read our CSV file and create a chart grouped by `group` and `gender`.  It requires a bit of transformation because we need to define a data series for each gender defined bar a `Bar` element, where `x` is a sequence of the group names and `y` is a sequence of total premiums for that group.  The name of each data series is the gender.

In [3]:
#r "nuget: FSharp.Data"
open FSharp.Data

type PoliciesProvider = CsvProvider<Sample="PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision", Culture="da-DK", Separators=";", Schema="Premium=decimal, TechnicalProvision=decimal">
let policies = PoliciesProvider.Load("https://raw.githubusercontent.com/t4rzsan/fsharp-for-actuaries/master/Policies.csv").Rows

let groupedLayout = Layout(barmode = "group")

policies
|> Seq.groupBy (fun p -> p.Gender)
|> Seq.map (fun (gender, rows) -> 
            // Generate tuple with gender and sequence of tuples with (group, premium)
            gender, 
            rows 
            |> Seq.groupBy (fun row -> row.Group)
            |> Seq.map (fun (group, groupRows) -> 
                        group, 
                        groupRows 
                        |> Seq.sumBy (fun groupRow -> groupRow.Premium))
           )
|> Seq.map (fun (gender, data) ->
                Bar(
                    x = (data |> Seq.map (fun (group, _) -> group)),
                    y = (data |> Seq.map (fun (_, premium) -> premium)),
                    name = gender
                )
           )
|> Chart.Plot
|> Chart.WithLayout groupedLayout
|> Chart.WithHeight 300
|> Chart.WithWidth 700


Installed package FSharp.Data version *

=============== S T A R T ==========================================
>>>> C:\Users\Jakob\.nuget\packages\fsharp.data\3.3.3\typeproviders\fsharp41\netstandard2.0\FSharp.Data.DesignTime.dll
>>>> C:\Users\Jakob\.nuget\packages\fsharp.data\3.3.3\lib\netstandard2.0\FSharp.Data.DesignTime.dll
Using: C:\Users\Jakob\.nuget\packages\fsharp.data\3.3.3\typeproviders\fsharp41\netstandard2.0\FSharp.Data.DesignTime.dll


This way of doing it is still correct if more groups or genders are added to the CSV file.  However, it is a pain to read and understand.  Creating helper functions may help on readability and also testability if you write unit tests.  Let us define a couple of helper functions.

In [4]:
let createGroupPremiums (group, groupRows: PoliciesProvider.Row seq) =
    group, 
    groupRows 
    |> Seq.sumBy (fun groupRow -> groupRow.Premium)
    
let createChartBar (gender, data) = 
    Bar(
        x = (data |> Seq.map (fun (group, _) -> group)),
        y = (data |> Seq.map (fun (_, premium) -> premium)),
        name = gender
    )
    
policies
|> Seq.groupBy (fun p -> p.Gender)
|> Seq.map (fun (gender, rows) -> 
            // Generate tuple with gender and sequence of tuples with (group, premium)
            gender, 
            rows 
            |> Seq.groupBy (fun row -> row.Group)
            |> Seq.map createGroupPremiums
           )
|> Seq.map createChartBar
|> Chart.Plot
|> Chart.WithLayout groupedLayout
|> Chart.WithHeight 300
|> Chart.WithWidth 700


Now, the intent of the code is hopefully clearer.

Did you notice that I used parentheses in the argument list for the two functions in lines 1 and 6, even though I told you earlier that function argument lists are not surrounded by parentheses in F#.  Well, actually both functions each have one argument only, and that argument is a tuple type and often tuple types are surrounded with parentheses.  You could also define `createCharBar` like below which would make it clearer that there is actually only on argument.

```fsharp
let createChartBar genderData = 
    // Use pattern matching to deconstruct the tuple.
    let (gender, data) = genderData
    Bar(
        x = (data |> Seq.map (fun (group, _) -> group)),
        y = (data |> Seq.map (fun (_, premium) -> premium)),
        name = gender
    )```
    
In general it is a easier and clearer to use record types or anonymous types instead of tuples.  But the charts require tuples so that is why we used tuples above.